# Experiment with finding circles in the Rett Girls videos

July 2018

This attempts using boundaries to find blobs for later processing

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 300

In [ ]:
vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'
vc = cv2.VideoCapture(vid)
vc.get(cv2.CAP_PROP_FRAME_COUNT), vc.get(cv2.CAP_PROP_FPS), vc.get(cv2.CAP_PROP_FRAME_WIDTH), vc.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])

In [ ]:
def grabFrame(fn, lab=True):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    if lab:
        im = cv2.cvtColor(im.astype(np.float32)/255.0, cv2.COLOR_BGR2LAB)
    else:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im
show(grabFrame(2200))
def toRGB(im):
    return (255 * cv2.cvtColor(im, cv2.COLOR_LAB2RGB)).astype(np.uint8)


In [ ]:
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)

im = grabFrame(1700)
pblue = isBlue(im)
show(pblue)
dblue = cv2.dilate(pblue, np.ones((3,3)), iterations=3)
show(dblue)
_, tblue = cv2.threshold(dblue, 0.8, 255, cv2.THRESH_BINARY)
show(tblue)

In [ ]:
def isCircular(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    #return 0.7 <= circularity <= 1.2
    return 0.5 <= circularity <= 1.4

def findBlobs(image, verbose=True):
    # find the special blue
    blue = isBlue(image)
    _, blue = cv2.threshold(blue, 0.6, 255, cv2.THRESH_BINARY)
    blue = blue.astype(np.uint8)
    # dilate a bit to fill in noise
    blue = cv2.dilate(blue, np.ones((3,3), dtype=np.uint8), iterations=1)
    blue = cv2.erode(blue, np.ones((3,3), dtype=np.uint8), iterations=1)
    
    if verbose:
        show(blue)
    # get the contours of the blue regions
    im2, contours, hierarchy = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if verbose:
        oim = image.copy()
        cv2.drawContours(oim, contours, -1, (0,0,0), 1)
        show(oim)
    # filter by area
    if True:
        minArea = np.pi * 4**2
        contours = [contour for contour in contours if minArea < cv2.contourArea(contour)]
        if verbose:
            oim = image.copy()
            cv2.drawContours(oim, contours, -1, (0,0,0), 1)
            show(oim)
    if False:
        # filter by circularity
        contours = [contour for contour in contours if isCircular(contour)]
        if verbose:
            oim = image.copy()
            cv2.drawContours(oim, contours, -1, (0,0,0), 1)
            show(oim)

    return contours
t = findBlobs(grabFrame(2200))

In [ ]:
t[0]

## Find all the blobs and save them away

In [ ]:
im = grabFrame(1700)
plt.imshow((255 * cv2.cvtColor(im, cv2.COLOR_LAB2RGB)).astype(np.uint8))

In [ ]:
results = []
nframes = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
for fno in range(100, 3680):
    if fno % 100 == 0:
        print(fno, len(results))
    frame = grabFrame(fno)
    for blob in findBlobs(frame, False):
        x, y, w, h = cv2.boundingRect(blob)
        pixels = frame[y:y+h, x:x+w].copy()
        results.append((fno, blob, pixels))

    

## Eliminate duplicates

In [ ]:
def rms(v):
    return np.sqrt(np.mean(v**2))

compressed = []
for f1, c1, p1 in results:
    for f2, c2, p2 in compressed:
        if c1.shape == c2.shape and rms(c1 - c2) < 1 and p1.shape == p2.shape and rms(p1 - p2) < 10:
            break
    else:
        compressed.append((f1, c1, p1))

In [ ]:
print(len(results), len(compressed))

In [ ]:
import pickle
pickle.dump(compressed, open('MSB_Video_1.blobs', 'wb'))